I have augmented the images that I will be classifying.  I created a function to resize, grayscale, and normalize the pixel values of each image.  Then I created a dataframe combining the images with their labels.  This uncovered the fact that about 9,000 of the 50,000 images were mislabeled.  The jpg name of the image could not find a partner jpg name in the labels csv provided.

Using the reduced data set, I split the data into stratified test and validation sets and saved them to disk as numpy arrays. The data started at over 11 Gbs in size which caused RAM issues on my local machine of only 16 Gb RAM. I could load the data but would run out of memory when trying to manipulate it. I solved this by using Google Colab Pro, which provides me with 25 GB of RAM.  This is still not enough to manipulate the data as much as I would like, but I have been working around it by making one change and saving it, then uploading that data to a new notebook with a fresh 25 GB to start.  My final training data is about 9 GB in size.  I may need to augment the images further through rotations and/or larger sizes to get more detail, but that will require even more RAM so I will start with the 9GB set.

I have created a base model with 94% training accuracy but only 67% validation accuracy.  The model is overfitting, but it gives me a starting point.

Changing leaky Relu to Relu improved validation accuracy to 70%

Dense layer neurons from 3,000 to 300 reduced validation accuracy to 69%

Reducing layers from 3 to 2 convolutional and pooling reduced validation accuracy to 65%




In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy import asarray
import tensorflow as tf
from tensorflow import keras

In [2]:
X_train=np.load('drive/MyDrive/X_train_strat.npy')

In [3]:
y_train=np.load('drive/MyDrive/y_train_strat.npy')

In [4]:
X_val=np.load('drive/MyDrive/X_val_strat.npy')

In [5]:
y_val=np.load('drive/MyDrive/y_val_strat.npy')

In [7]:
X_train.shape, y_train.shape

((33089, 150, 200), (33089, 28))

In [6]:
## Baseline Model

# from tensorflow.keras import layers
# from keras.models import Sequential,Input,Model
# from keras.layers import Dense, Dropout, Flatten
# from keras.layers import Conv2D, MaxPooling2D
# from keras.layers.advanced_activations import LeakyReLU

# model = keras.models.Sequential()
# model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',input_shape=(150,200,1),padding='same'))
# model.add(LeakyReLU(alpha=0.1))
# model.add(MaxPooling2D((2, 2),padding='same'))
# model.add(Conv2D(64, (3, 3), activation='linear',padding='same'))
# model.add(LeakyReLU(alpha=0.1))
# model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
# model.add(Conv2D(128, (3, 3), activation='linear',padding='same'))
# model.add(LeakyReLU(alpha=0.1))                  
# model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
# model.add(Flatten())
# model.add(Dense(3000, activation='linear'))
# model.add(LeakyReLU(alpha=0.1))                  
# model.add(Dense(28, activation='softmax'))

In [7]:
## model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
## model.fit(X_train, y_train,batch_size=30, epochs=5)

Epoch 1/5
1103/1103 [==============================] - 56s 39ms/step - loss: 1.4638 - accuracy: 0.5738
Epoch 2/5
1103/1103 [==============================] - 43s 39ms/step - loss: 0.9173 - accuracy: 0.7120
Epoch 3/5
1103/1103 [==============================] - 44s 39ms/step - loss: 0.5395 - accuracy: 0.8241
Epoch 4/5
1103/1103 [==============================] - 43s 39ms/step - loss: 0.2643 - accuracy: 0.9122
Epoch 5/5
1103/1103 [==============================] - 43s 39ms/step - loss: 0.1778 - accuracy: 0.9439


In [11]:
##model.evaluate(X_val,y_val)

259/259 [==============================] - 3s 11ms/step - loss: 1.8553 - accuracy: 0.6786


[1.855276346206665, 0.6785930395126343]

###### Baseline model not stratified.  Training accuracy 0.95   evaluation accuracy 0.65
##### Stratified Baseline model       Training accuracy 0.94   evaluation accuracy 0.67
##### Stratified Relu, not leaky relu Training accuracy 0.96   evaluation accuracy 0.70

### Model2  not leaky Relu

In [10]:
from tensorflow.keras import layers
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

model2 = keras.models.Sequential()
model2.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(150,200,1),padding='same'))
model2.add(MaxPooling2D((2, 2),padding='same'))
model2.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
model2.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model2.add(Conv2D(128, (3, 3), activation='relu',padding='same'))                 
model2.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model2.add(Flatten())
model2.add(Dense(3000, activation='relu'))               
model2.add(Dense(28, activation='softmax'))

In [11]:
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
model2.fit(X_train, y_train,batch_size=30, epochs=5)

InternalError: ignored

In [10]:
model2.evaluate(X_val,y_val)

259/259 [==============================] - 3s 11ms/step - loss: 1.6980 - accuracy: 0.7037


[1.6980047225952148, 0.7037350535392761]

### Model3

In [6]:
from tensorflow.keras import layers
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

model3 = keras.models.Sequential()
model3.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(150,200,1),padding='same'))
model3.add(MaxPooling2D((2, 2),padding='same'))
model3.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
model3.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model3.add(Conv2D(128, (3, 3), activation='relu',padding='same'))                 
model3.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model3.add(Flatten())
model3.add(Dense(300, activation='relu'))               
model3.add(Dense(28, activation='softmax'))

In [7]:
model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
model3.fit(X_train, y_train,batch_size=30, epochs=5)

Epoch 1/5
1103/1103 [==============================] - 45s 26ms/step - loss: 1.4102 - accuracy: 0.5813
Epoch 2/5
1103/1103 [==============================] - 28s 26ms/step - loss: 0.8974 - accuracy: 0.7198
Epoch 3/5
1103/1103 [==============================] - 28s 26ms/step - loss: 0.5801 - accuracy: 0.8146
Epoch 4/5
1103/1103 [==============================] - 28s 26ms/step - loss: 0.2879 - accuracy: 0.9052
Epoch 5/5
1103/1103 [==============================] - 28s 25ms/step - loss: 0.1295 - accuracy: 0.9584


In [9]:
model3.evaluate(X_val,y_val)

259/259 [==============================] - 3s 11ms/step - loss: 1.5444 - accuracy: 0.6983


[1.544400691986084, 0.6982956528663635]